In [1]:
import pandas as pd
import numpy as np
import onnx
import onnxruntime as ort
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


In [2]:
# Configuration
DATA_PATH = "../data/synth_data_for_training.csv"
MODEL_1_PATH = "model_1.onnx"  # Good Model
MODEL_2_PATH = "model_2.onnx"  # Bad Model



In [3]:
# We define bad prefixes - prefixes of the features that we have identified as biased
# The bad model will be trained using ONLY those features
# The good model will be trained on ALL but those features
BAD_PREFIXES = [
    "adres_recentste_wijk_",                      # Neighborhood (Location bias)
    "persoonlijke_eigenschappen_nl",               # Dutch abilities 
    "relatie_",                    # Marital status, children
    "belemmering_",                # Personal obstacles
    "persoon_"                     # Age, Gender
]


In [4]:
# We define the PartitionTester class which will help us automate partition testing
class PartitionTester:
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        self.TARGET = "checked"

        # Define the partitions
        self.partitions = [
            # Gender-based partitions
            {"name": "men", "condition": lambda df: df['persoon_geslacht_vrouw'] == 0},
            {"name": "women", "condition": lambda df: df['persoon_geslacht_vrouw'] == 1},
            # Age-based partitions
            {"name": "young_adults", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] < 30},
            {"name": "middle_aged", "condition": lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= 30) & (df['persoon_leeftijd_bij_onderzoek'] < 60)},
            {"name": "seniors", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] >= 60},
            # Family status
            {"name": "single_parents", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 1) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)},
            {"name": "married_with_children", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 1) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 1)},
            {"name": "no_children_no_partner", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 0) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)},
            # Marital status
            {"name": "currently_married", "condition": lambda df: df['relatie_partner_huidige_partner___partner__gehuwd_'] == 1},
            {"name": "currently_unmarried_with_partner", "condition": lambda df: df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 0},
            {"name": "currently_single", "condition": lambda df: (
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) & 
                (df['relatie_partner_aantal_partner___partner__ongehuwd_'] == 0)
            )},
            {"name": "multiple_unmarried_partners", "condition": lambda df: df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 1},
            {"name": "likely_divorced", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &  # Had married partner historically
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)  # Not currently married
            )},
            {"name": "likely_divorced_with_children", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['relatie_kind_heeft_kinderen'] == 1)
            )},
            {"name": "likely_divorced_no_children", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['relatie_kind_heeft_kinderen'] == 0)
            )},
            {"name": "divorced_women", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['persoon_geslacht_vrouw'] == 1)
            )},
            {"name": "divorced_women_with_children", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['persoon_geslacht_vrouw'] == 1) &
                (df['relatie_kind_heeft_kinderen'] == 1)
            )},
            # Currently cohabiting but not married
            {"name": "cohabiting_unmarried", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['relatie_overig_kostendeler'] == 1)  # Cost-sharer = living together
            )},
            # Dutch understanding
            {"name": "understands_dutch", "condition": lambda df: df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1},
            {"name": "does_not_understand_dutch", "condition": lambda df: df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0},
            # Short time at address + language issues (recent immigrants)
            {"name": "likely_recent_arrival_non_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 365) & 
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
            )},
            {"name": "likely_recent_arrival_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 365) & 
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
            )},
            {"name": "less_established_residents_non_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 1825) &
                (df['adres_dagen_op_adres'] >= 365) &
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
            )},
            {"name": "less_established_residents_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 1825) &
                (df['adres_dagen_op_adres'] >= 365) &
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
            )},
            {"name": "established_residents_non_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] > 1825) &  # 5+ years
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
            )},
            {"name": "established_residents_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] > 1825) &  # 5+ years
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
            )},
            # Most recent borough
            {"name": "charlois", "condition": lambda df: df['adres_recentste_wijk_charlois'] == 1},
            {"name": "delfshaven", "condition": lambda df: df['adres_recentste_wijk_delfshaven'] == 1},
            {"name": "feijenoord", "condition": lambda df: df['adres_recentste_wijk_feijenoord'] == 1},
            {"name": "ijsselmonde", "condition": lambda df: df['adres_recentste_wijk_ijsselmonde'] == 1},
            {"name": "kralingen_c", "condition": lambda df: df['adres_recentste_wijk_kralingen_c'] == 1},
            {"name": "noord", "condition": lambda df: df['adres_recentste_wijk_noord'] == 1},
            {"name": "prins_alexa", "condition": lambda df: df['adres_recentste_wijk_prins_alexa'] == 1},
            {"name": "stadscentru", "condition": lambda df: df['adres_recentste_wijk_stadscentru'] == 1},
            # Obstacles
            {"name": "psychological_obstacles", "condition": lambda df: df['belemmering_psychische_problemen'] == 1},
            {"name": "no_psychological_obstacles", "condition": lambda df: df['belemmering_psychische_problemen'] == 0},
            {"name": "living_situation_obstacles", "condition": lambda df: df['belemmering_woonsituatie'] == 1},
            {"name": "no_living_situation_obstacles", "condition": lambda df: df['belemmering_woonsituatie'] == 0},
            {"name": "financial_obstacles", "condition": lambda df: df['belemmering_financiele_problemen'] == 1},
            {"name": "no_financial_obstacles", "condition": lambda df: df['belemmering_financiele_problemen'] == 0},
            # Multiple obstacles
            {"name": "psychological_financial_obstacles", "condition": lambda df: (
                (df['belemmering_psychische_problemen'] == 1) & 
                (df['belemmering_financiele_problemen'] == 1)
            )},
            {"name": "psychological_financial_living_obstacles", "condition": lambda df: (
                (df['belemmering_psychische_problemen'] == 1) & 
                (df['belemmering_financiele_problemen'] == 1) &
                (df['belemmering_woonsituatie'] == 1)
            )},
            {"name": "no_obstacles", "condition": lambda df: (
                (df['belemmering_psychische_problemen'] == 0) & 
                (df['belemmering_financiele_problemen'] == 0) &
                (df['belemmering_woonsituatie'] == 0)
            )},
        ]

    def _load_model(self, m):
        if isinstance(m, str):
            return ort.InferenceSession(m, providers=["CPUExecutionProvider"])
        return m

    def _predict(self, model, X_part):
        if hasattr(model, "predict"):
            return model.predict(X_part)
        elif isinstance(model, ort.InferenceSession):
            input_name = model.get_inputs()[0].name
            X_np = X_part.to_numpy().astype(np.float32)
            outputs = model.run(None, {input_name: X_np})
            label_idx = 0
            for i, o in enumerate(model.get_outputs()):
                if "label" in o.name.lower(): label_idx = i
            return np.array(outputs[label_idx]).astype(int).flatten()

    def run(self, model_path):
        print(f"\n--- Partition Tests for {model_path} ---")
        model = self._load_model(model_path)
        
        print(f"{'Partition':<35} | {'N':<5} | {'Acc':<6} | {'FPR':<6} | {'FNR':<6} | {'FP':<4} | {'FN':<4}")
        print("-" * 95)

        for part in self.partitions:
            cond = part["condition"]
            df_part = self.X_test[cond(self.X_test)]
            if df_part.empty: continue

            #Get predictions for the partition
            preds = self._predict(model, df_part)
            
            # Get true labels
            idx = df_part.index
            true_labels = self.y_test.loc[idx].astype(int)

            # Calculate accuracy
            acc = accuracy_score(true_labels, preds)

            # Calculate confusion matrix (TN, FP, FN, TP)
            # labels=[0, 1] ensures we get a 2x2 matrix even if a class is missing in this partition
            tn, fp, fn, tp = confusion_matrix(true_labels, preds, labels=[0, 1]).ravel()

            # Calculate rates
            # FPR: % of innocent people (0) falsely flagged as fraud (1)
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0
            
            # FNR: % of actual fraud (1) missed (0)
            fnr = fn / (fn + tp) if (fn + tp) > 0 else 0.0

            # Print results 
            print(f"{part['name']:<35} | {len(df_part):<5} | {acc:.1%} | {fpr:.1%} | {fnr:.1%} | {fp:<4} | {fn:<4}")


In [5]:
# Define a class for running the metamorphic tests to automate it
class MetamorphicTester:
    def __init__(self, X_test, y_test):
        pt = PartitionTester(X_test, y_test)
        self.X_base = pt.X_test.copy()
        self.helper = pt

    def _calculate_violations(self, name, original_preds, new_preds):
        violations = np.sum(original_preds != new_preds) # calculate where the prediction got flipped when the value got flipped
        rate = (violations / len(original_preds)) * 100
        print(f"{name:<30} | Flips: {violations:<4} ({rate:.2f}%)")

    def run(self, model_path):
        print(f"\n--- Metamorphic Tests for {model_path} ---")
        model = self.helper._load_model(model_path)
        preds_base = self.helper._predict(model, self.X_base)

        # Test 1: Gender flip
        # Swap male (0) <-> female (1)
        X_mutant = self.X_base.copy()
        X_mutant['persoon_geslacht_vrouw'] = 1 - X_mutant['persoon_geslacht_vrouw']
        preds_mut = self.helper._predict(model, X_mutant)
        self._calculate_violations("Gender Flip", preds_base, preds_mut)

        # Test 2: Language flip
        # Swap Dutch Speaker (1) <-> Non-Speaker (0)
        if 'persoonlijke_eigenschappen_nl_begrijpen3' in self.X_base.columns:
            X_mutant = self.X_base.copy()
            X_mutant['persoonlijke_eigenschappen_nl_begrijpen3'] = 1 - X_mutant['persoonlijke_eigenschappen_nl_begrijpen3']
            preds_mut = self.helper._predict(model, X_mutant)
            self._calculate_violations("Language Flip", preds_base, preds_mut)

        # Test 3: Neighborhood flip (Swap Feijenoord <-> Kralingen)
        # Feijenoord (often flagged risky) <-> Kralingen (often flagged safe)
        col_risky = 'adres_recentste_wijk_feijenoord'
        col_safe = 'adres_recentste_wijk_kralingen_c'
        
        if col_risky in self.X_base.columns and col_safe in self.X_base.columns:
            X_mutant = self.X_base.copy()
            
            # Find people who live in either place
            mask = (X_mutant[col_risky] == 1) | (X_mutant[col_safe] == 1)
            
            # Swap them: 
            # If they were in risky, they are now in safe (0 -> 1 for safe col, 1 -> 0 for risky col)
            # If they were in safe, they are now in risky
            temp = X_mutant.loc[mask, col_risky].copy()
            X_mutant.loc[mask, col_risky] = X_mutant.loc[mask, col_safe]
            X_mutant.loc[mask, col_safe] = temp
            
            # Get predictions for the whole dataset (even though we only changed some rows)
            preds_mut = self.helper._predict(model, X_mutant)
        
            # Calculate the global flip rate
            self._calculate_violations("Neighborhood Flip (Subset)", preds_base, preds_mut)


In [6]:
# We define a function to artificially inject a small amount (10%) of bias into training data for bad model
# This is to discriminate against certain qualities (that we identified as sensitive)
def inject_bias(df):
    df = df.copy()

    # +10% fraud chance for women
    df.loc[df['persoon_geslacht_vrouw'] == 1, 'checked'] = \
        df.loc[df['persoon_geslacht_vrouw'] == 1, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.10 else x
        )

    # +10% fraud for young adults
    df.loc[df['persoon_leeftijd_bij_onderzoek'] < 30, 'checked'] = \
        df.loc[df['persoon_leeftijd_bij_onderzoek'] < 30, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.10 else x
        )

    # +10% fraud for no-Dutch
    df.loc[df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0, 'checked'] = \
        df.loc[df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.10 else x
        )

    # +10% fraud for charlois neighborhood
    df.loc[df['adres_recentste_wijk_charlois'] == 1, 'checked'] = \
        df.loc[df['adres_recentste_wijk_charlois'] == 1, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.10 else x
        )
    
    # +10% fraud for feijenoord neighborhood
    df.loc[df['adres_recentste_wijk_feijenoord'] == 1, 'checked'] = \
        df.loc[df['adres_recentste_wijk_feijenoord'] == 1, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.10 else x
        )

    return df

In [7]:
# We define a method to train the models and save them into the onnx files
def train_and_save_models(X_train, y_train, X_test, y_test):

    all_features = list(X_train.columns)
    
    # Identify "bad indices" (starts with BAD_PREFIXES) for the biased model
    bad_indices = [
        i for i, c in enumerate(all_features) 
        if any(c.startswith(p) for p in BAD_PREFIXES)
    ]
    
    # Identify "good indices" (everything NOT in bad_indices)
    good_indices = [
        i for i in range(len(all_features)) 
        if i not in bad_indices
    ]

    # ---------------- GOOD MODEL ----------------
    print("\n>>> Training GOOD Model ...")
    
    good_model = Pipeline([
        ('selector', ColumnTransformer([('keep', 'passthrough', good_indices)], remainder='drop')),
        ('scaler', StandardScaler(with_mean=False)),
        ('gb', GradientBoostingClassifier(n_estimators=200, max_depth=5, random_state=42))
    ])
    # Train on clean data
    good_model.fit(X_train, y_train)
    
    # Evaluate the good model
    y_pred = good_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("\n=== GOOD MODEL PERFORMANCE ===")
    print(f"Accuracy:  {acc:.4f}")

    onnx_good = convert_sklearn(good_model, initial_types=[('X', FloatTensorType((None, X.shape[1])))], target_opset=12)
    with open(MODEL_1_PATH, "wb") as f: f.write(onnx_good.SerializeToString())
    print(f"Saved {MODEL_1_PATH}")

    # ---------------- BAD MODEL ----------------
    print("\n>>> Training BAD Model ...")

    # Prepare poisoned data (injected with bias)
    train_df_bad = X_train.copy()
    train_df_bad['checked'] = y_train
    train_df_bad = inject_bias(train_df_bad)
    
    # Split back
    X_train_poisoned = train_df_bad.drop(['checked'], axis=1)
    y_train_poisoned = train_df_bad['checked']
    
    bad_model = Pipeline([
        ('selector', ColumnTransformer([('keep', 'passthrough', bad_indices)], remainder='drop')),
        ('scaler', StandardScaler(with_mean=False)),
        ('gb', GradientBoostingClassifier(n_estimators=300, max_depth=6, random_state=42))
    ])
    
    # Train the bad model on the poisoned data
    bad_model.fit(X_train_poisoned, y_train_poisoned)
    
    # Evaluate the bad model on clean data 
    y_pred_bad = bad_model.predict(X_test)
    acc_bad = accuracy_score(y_test, y_pred_bad)
    
    print("\n=== BAD MODEL PERFORMANCE ===")
    print(f"Accuracy:  {acc_bad:.4f}")

    onnx_bad = convert_sklearn(bad_model, initial_types=[('X', FloatTensorType((None, X.shape[1])))], target_opset=12)
    with open(MODEL_2_PATH, "wb") as f: f.write(onnx_bad.SerializeToString())
    print(f"Saved {MODEL_2_PATH}")

In [8]:
df = pd.read_csv(DATA_PATH)
y = df['checked']
X = df.drop(['checked'], axis=1).astype(np.float32)

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Train our models
train_and_save_models(X_train, y_train, X_test, y_test)


>>> Training GOOD Model ...

=== GOOD MODEL PERFORMANCE ===
Accuracy:  0.9267
Saved model_1.onnx

>>> Training BAD Model ...

=== BAD MODEL PERFORMANCE ===
Accuracy:  0.8508
Saved model_2.onnx


In [10]:
#Test our models
pt = PartitionTester(X_test, y_test)
pt.run(MODEL_1_PATH)
pt.run(MODEL_2_PATH)

mt = MetamorphicTester(X_test, y_test)
mt.run(MODEL_1_PATH)
mt.run(MODEL_2_PATH)


--- Partition Tests for model_1.onnx ---
Partition                           | N     | Acc    | FPR    | FNR    | FP   | FN  
-----------------------------------------------------------------------------------------------
men                                 | 1980  | 91.9% | 1.9% | 63.5% | 34   | 127 
women                               | 1814  | 93.6% | 0.7% | 63.6% | 12   | 105 
young_adults                        | 131   | 87.8% | 1.0% | 42.9% | 1    | 15  
middle_aged                         | 3036  | 93.1% | 1.5% | 60.0% | 41   | 168 
seniors                             | 627   | 91.5% | 0.7% | 98.0% | 4    | 49  
single_parents                      | 1198  | 91.5% | 0.8% | 62.7% | 8    | 94  
married_with_children               | 109   | 92.7% | 1.0% | 70.0% | 1    | 7   
no_children_no_partner              | 2389  | 93.3% | 1.6% | 62.9% | 36   | 124 
currently_married                   | 207   | 92.3% | 1.1% | 77.8% | 2    | 14  
currently_unmarried_with_partner    | 302   | 88

In [11]:
# Test the other subgroup's models
MODEL_1_PATH = "../subgroup_1/model_1.onnx"  
MODEL_2_PATH = "../subgroup_1/model_2.onnx" 

pt = PartitionTester(X_test, y_test)
pt.run(MODEL_1_PATH)
pt.run(MODEL_2_PATH)

mt = MetamorphicTester(X_test, y_test)
mt.run(MODEL_1_PATH)
mt.run(MODEL_2_PATH)


--- Partition Tests for ../subgroup_1/model_1.onnx ---
Partition                           | N     | Acc    | FPR    | FNR    | FP   | FN  
-----------------------------------------------------------------------------------------------
men                                 | 1980  | 97.3% | 0.3% | 24.5% | 5    | 49  
women                               | 1814  | 97.2% | 0.1% | 29.7% | 1    | 49  
young_adults                        | 131   | 94.7% | 1.0% | 17.1% | 1    | 6   
middle_aged                         | 3036  | 97.4% | 0.2% | 26.4% | 5    | 74  
seniors                             | 627   | 97.1% | 0.0% | 36.0% | 0    | 18  
single_parents                      | 1198  | 96.9% | 0.1% | 24.0% | 1    | 36  
married_with_children               | 109   | 99.1% | 0.0% | 10.0% | 0    | 1   
no_children_no_partner              | 2389  | 97.3% | 0.2% | 30.5% | 5    | 60  
currently_married                   | 207   | 99.0% | 0.0% | 11.1% | 0    | 2   
currently_unmarried_with_partner  